In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

In [4]:
# !cp cfg/yolov3.cfg cfg/yolov3_training.cfg
!cp cfg/yolov3-tiny.cfg cfg/yolov3-tiny_training.cfg

In [5]:
# max_batches = classes * 2000 (3 classes = 6000)
# !sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
# !sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
# !sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_training.cfg

!sed -i 's/batch=1/batch=64/' cfg/yolov3-tiny_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3-tiny_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 8000/' cfg/yolov3-tiny_training.cfg

# classes = num classes
# filters = (classes + 5) * 3 ((3 classes + 5) * 3 = 24)
# !sed -i '610 s@classes=80@classes=3@' cfg/yolov3_training.cfg
# !sed -i '696 s@classes=80@classes=3@' cfg/yolov3_training.cfg
# !sed -i '783 s@classes=80@classes=3@' cfg/yolov3_training.cfg
# !sed -i '603 s@filters=255@filters=24@' cfg/yolov3_training.cfg
# !sed -i '689 s@filters=255@filters=24@' cfg/yolov3_training.cfg
# !sed -i '776 s@filters=255@filters=24@' cfg/yolov3_training.cfg

!sed -i '610 s@classes=80@classes=4@' cfg/yolov3-tiny_training.cfg
!sed -i '696 s@classes=80@classes=4@' cfg/yolov3-tiny_training.cfg
!sed -i '783 s@classes=80@classes=4@' cfg/yolov3-tiny_training.cfg
!sed -i '603 s@filters=255@filters=27@' cfg/yolov3-tiny_training.cfg
!sed -i '689 s@filters=255@filters=27@' cfg/yolov3-tiny_training.cfg
!sed -i '776 s@filters=255@filters=27@' cfg/yolov3-tiny_training.cfg

In [6]:
# don't forget to change the class
!echo -e "stem_grapevine_L\nstem_grapevine_M\nstem_grapevine_S\nnot_grapevine_its_Chili" > data/obj.names
!echo -e 'classes= 4\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

In [7]:
# !cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg
# !cp data/obj.names /mydrive/yolov3/classes.txt

!cp cfg/yolov3-tiny_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

In [ ]:
!mkdir data/obj
!unzip /mydrive/yolov3/images.zip -d data/obj

In [9]:
import glob
image_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
  f.write("\n".join(image_list))

In [10]:
# yolov3
# !wget https://pjreddie.com/media/files/darknet53.conv.74

# tiny
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

--2023-07-23 19:20:03--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M  12.0MB/s    in 2.8s    

2023-07-23 19:20:07 (12.0 MB/s) - ‘yolov3-tiny.weights’ saved [35434956/35434956]



In [11]:
# Start the training
# !./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

# tiny && if connection aborted use yolov3_training_last.weights
!./darknet partial cfg/yolov3-tiny_training.cfg yolov3-tiny.weights yolov3-tiny.conv.15 15
!./darknet detector train data/obj.data cfg/yolov3-tiny_training.cfg yolov3-tiny.conv.15 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.553901), count: 4, class_loss = 0.585549, iou_loss = 0.736222, total_loss = 1.321770 
 total_bbox = 1161225, rewritten_bbox = 0.057525 % 
]2;7903/8000: loss=1.3 hours left=0.1
 7903: 1.261591, 1.278494 avg loss, 0.001000 rate, 2.390484 seconds, 505792 images, 0.089762 hours left
Loaded: 0.082587 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.729673), count: 9, class_loss = 1.594032, iou_loss = 0.805728, total_loss = 2.399760 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.601036), count: 2, class_loss = 0.435023, iou_loss = 0.291960, total_loss = 0.726982 
 total_bbox = 1161236, rewritten_bbox = 0.057525 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.773919), count: 4, class_loss = 0.572746, iou_loss = 0.321175, total_loss = 0.893921 
v3 (